In [1]:


# !pip install ultralytics





In [8]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='TRUE'
import ultralytics
from ultralytics import YOLO
import cv2 as cv
import time
# import fastreid.modeling.heads 
model = YOLO("yolov8n.pt")
model2=YOLO("yolo11n.pt")
model3=YOLO("yolov8m.pt")
model4=YOLO("yolov8x.pt")


# Your other imports like numpy, pandas, etc. go here

In [3]:
# !pip install opencv-python

In [4]:
# !pip install chardet

In [9]:
import cv2 as cv
import torch
import torchvision.transforms as T
from PIL import Image # <--- Import the Image class from Pillow

# --- 1. Load the image using OpenCV ---

def transform(img):
# --- 2. Define the Transformation Pipeline (no changes here) ---
    image_height = 256
    image_width = 128
    pixel_mean = [0.485, 0.456, 0.406]
    pixel_std = [0.229, 0.224, 0.225]

    transform = T.Compose([
        T.Resize((image_height, image_width)),
        T.ToTensor(),
        T.Normalize(mean=pixel_mean, std=pixel_std)
    ])

    # --- 3. Apply the Transformations ---

    # a. Convert color from BGR (OpenCV) to RGB
    rgb_crop = cv.cvtColor(img, cv.COLOR_BGR2RGB)


    # b. Convert the NumPy array to a PIL Image  <--- THIS IS THE FIX
    pil_image = Image.fromarray(rgb_crop)

    # c. Apply the preprocessing pipeline to the PIL Image
    tensor = transform(pil_image)

    # d. Add the batch dimension
    input_batch = tensor.unsqueeze(0)

    # e. Move to the correct device
    device = "cuda" if torch.cuda.is_available() else "cpu"
    input_batch = input_batch.to(device)
    
    
    return input_batch

In [14]:

import torch
import os
from pathlib import Path


import sys
sys.path.append('fast-reid')

from fastreid.config import get_cfg
from fastreid.modeling import build_model
from fastreid.utils.checkpoint import Checkpointer
import fastreid.modeling.heads 
import fastreid.modeling.backbones
def setup_cfg(config_file, weights_file):
    """
    Creates a FastReID config object and merges it with the specified
    configuration file and weights.
    """
    # 1. Get a copy of the default configuration
    cfg = get_cfg()

    # 2. Add project-specific configurations if any
    # (Not needed for this baseline model)

    # 3. Merge from the specified YAML configuration file
    # This file contains the model architecture and dataset specifics.
    cfg.merge_from_file(config_file)

    # 4. Set the path to the pre-trained model weights.
    # This is a critical step for loading the desired model state.
    cfg.MODEL.WEIGHTS = weights_file

    # 5. Set the device to use for inference.
    # Use 'cuda' for GPU or 'cpu' for CPU.
    cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    cfg.MODEL.HEADS.NUM_CLASSES = 751
    # 6. Freeze the configuration.
    # This makes the config object immutable and prevents accidental changes.
    cfg.freeze()
    
    return cfg

def main():
    """
    Main function to load the model and print its architecture.
    """
    # --- Configuration ---
    # Define paths relative to the project structure.
    project_root = Path('.')
    fastreid_repo_path = project_root / 'fast-reid'
    
    # Path to the configuration YAML file.
    config_file_path = fastreid_repo_path / 'configs/Market1501/bagtricks_R101-ibn.yml'
    
    # Path to the pre-trained weights file.
    weights_file_path = project_root / "C:/pooo/models/market_bot_R101-ibn.pth"

    # Verify that the necessary files exist.
    if not config_file_path.is_file():
        print(f"Error: Config file not found at {config_file_path}")
        return
    if not weights_file_path.is_file():
        print(f"Error: Weights file not found at {weights_file_path}")
        print("Please download 'market_bot_R50.pth' and place it in the 'models/' directory.")
        return

    # --- Model Loading ---
    # 1. Setup the configuration object.
    cfg = setup_cfg(str(config_file_path), str(weights_file_path))
    print(f"Configuration loaded successfully. Using device: {cfg.MODEL.DEVICE}")

    # 2. Build the model.
    # The `build_model` function reads the configuration and constructs the
    # corresponding model architecture. It also automatically loads the weights
    # specified in `cfg.MODEL.WEIGHTS`.
    model = build_model(cfg)
    
    print("Model built successfully.")

    # The Checkpointer is used internally by `build_model` but can also be
    # used explicitly if needed. For this use case, `build_model` handles it.
    # Checkpointer(model).load(cfg.MODEL.WEIGHTS) # This is done inside build_model

    # 3. Set the model to evaluation mode.
    # This is crucial for consistent results during inference as it disables
    # layers like Dropout and uses running statistics for Batch Normalization.
    model.eval()
    print("Model set to evaluation mode.")

    # --- Robust Weight Verification ---
   # --- Manual Weight Correction and Loading ---
    print("\n--- Manually Correcting and Loading Weights ---")
    
    device = torch.device(cfg.MODEL.DEVICE)
    weights_from_file = torch.load(weights_file_path, map_location=device)
    
    # Check if weights are nested under a 'model' key
    if 'model' in weights_from_file:
        weights_from_file = weights_from_file['model']

    # Create a new dictionary to hold the corrected weights
    from collections import OrderedDict
    corrected_weights = OrderedDict()
    
    # Iterate through the keys from the file and rename them
    for k, v in weights_from_file.items():
        # --- THIS IS THE FIX ---
        if k.startswith('module.'):
             # Remove 'module.' prefix if it exists
            new_key = k[7:]
        else:
            new_key = k
        
        # Rename the head layers
        if new_key == 'heads.classifier.weight':
            new_key = 'heads.weight'
        elif 'heads.bnneck.' in new_key:
            new_key = new_key.replace('heads.bnneck.', 'heads.bottleneck.0.')
        
        corrected_weights[new_key] = v
        # --------------------

    # Load the corrected weights into the model.
    # We use strict=False to ignore keys that are in the file but not needed by the model
    # (like pixel_mean, pixel_std).
    incompatible = model.load_state_dict(corrected_weights, strict=False)
    
    if not incompatible.missing_keys and not incompatible.unexpected_keys:
        print("✅ Success! All model weights were loaded correctly after manual correction.")
    else:
        print("❌ Error! Still some mismatches after correction.")
        print(f"Missing keys in model: {incompatible.missing_keys}")
        print(f"Unexpected keys in file: {incompatible.unexpected_keys}")

    print("--------------------------------\n")

    # --- Verification ---
    # ... (the rest of the script, like printing the model architecture) ...

    # --- Verification ---
    # Print the model architecture to verify it has been loaded correctly.
    # You should see the ResNet-50 blocks, the pooling layer, and the BNNeck.
    print("\n--- Model Architecture ---")
    print(model)
    print("--------------------------\n")
    
    print("FastReID model is ready for feature extraction.")
    
    # The 'model' object is now ready to be used for inference.
    return model

if __name__ == '__main__':
    loaded_model = main()

Configuration loaded successfully. Using device: cuda
Model built successfully.
Model set to evaluation mode.

--- Manually Correcting and Loading Weights ---
❌ Error! Still some mismatches after correction.
Missing keys in model: []
Unexpected keys in file: ['pixel_mean', 'pixel_std']
--------------------------------


--- Model Architecture ---
Baseline(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): IBN(
          (IN): InstanceNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
          (BN): BatchNorm(32, eps=1e-05, momentum=0.1, affine=True, trac

In [15]:
import cv2 as cv
import torch
import time
import csv
import os
from collections import deque, defaultdict
from ultralytics import YOLO
from torchvision import transforms

# --- Parameters ---
SIMILARITY_THRESHOLD = 0.92
FEATURE_HISTORY = 30
CONFIRM_FRAMES = 8
MIN_BOX_AREA = 800
REAPPEAR_THRESHOLD = 2.0  # seconds

# --- Data structures ---
pending_tracks = defaultdict(lambda: {"count": 0, "feature": None})
tracked_identities = defaultdict(lambda: deque(maxlen=FEATURE_HISTORY))
tracker_to_reid = {}
next_permanent_id = 0

# --- Appearance/reappearance logging structures ---
last_seen_time = {}  # permanent_id -> last timestamp
appearance_log = defaultdict(list)  # permanent_id -> [(event_type, timestamp)]

# --- CSV logging setup ---
csv_file_path = os.path.abspath("person_appearance_log.csv")
write_header = not os.path.exists(csv_file_path)

csv_file = open(csv_file_path, mode="a", newline="")
csv_writer = csv.writer(csv_file)

if write_header:
    csv_writer.writerow(["PersonID", "EventType", "Time_HHMMSS"])
    csv_file.flush()

# --- Best match function ---
def best_match(feature, stored_features):
    best_id = -1
    best_score = -1
    for pid, feats in stored_features.items():
        feats_tensor = torch.stack(list(feats))
        max_sim = torch.nn.functional.cosine_similarity(feature, feats_tensor).max().item()
        avg_feat = feats_tensor.mean(dim=0, keepdim=True)
        avg_sim = torch.nn.functional.cosine_similarity(feature, avg_feat).item()
        final_score = 0.85 * max_sim + 0.15 * avg_sim
        if final_score > best_score:
            best_score = final_score
            best_id = pid
    return best_id, best_score

# --- Crop preprocessing ---
def preprocess_crop(frame, box):
    x1, y1, x2, y2 = map(int, box)
    crop = frame[y1:y2, x1:x2]
    if crop.size == 0 or (x2-x1)*(y2-y1) < MIN_BOX_AREA:
        return None
    return transform(crop)  # Make sure transform is defined

# --- Main loop ---
ptime = 0
for result in model3.track(source="palace.mp4", stream=True, tracker="bytetrack.yaml", classes=[0]):
    frame = result.orig_img
    current_time = time.time()

    if result.boxes.id is None:
        cv.imshow("Person Re-ID Tracking", frame)
        if cv.waitKey(1) == ord('q'):
            break
        continue

    boxes = result.boxes.xyxy.cpu().numpy()
    tracker_ids = result.boxes.id.cpu().numpy().astype(int)

    crops_to_match = []
    id_mapping = []

    # Step 1: Collect crops for unmatched tracker IDs
    for box, tracker_id in zip(boxes, tracker_ids):
        if tracker_id not in tracker_to_reid:
            crop_tensor = preprocess_crop(frame, box)
            if crop_tensor is not None:
                crops_to_match.append(crop_tensor)
                id_mapping.append(tracker_id)

    # Step 2: Batch ReID for new/unmatched tracker IDs
    if crops_to_match:
        batch_tensor = torch.cat(crops_to_match, dim=0)
        with torch.no_grad():
            features_batch = loaded_model(batch_tensor)
        for tracker_id, feature in zip(id_mapping, features_batch):
            best_id, sim = best_match(feature, tracked_identities)
            if sim > SIMILARITY_THRESHOLD:
                tracker_to_reid[tracker_id] = best_id
                tracked_identities[best_id].append(feature)
            else:
                pending_tracks[tracker_id]["count"] += 1
                if pending_tracks[tracker_id]["count"] >= CONFIRM_FRAMES:
                    global next_permanent_id
                    new_id = next_permanent_id
                    next_permanent_id += 1
                    tracker_to_reid[tracker_id] = new_id
                    tracked_identities[new_id].append(feature)
                    del pending_tracks[tracker_id]
                else:
                    pending_tracks[tracker_id]["feature"] = feature

    # Step 3: Drawing + CSV logging
    for box, tracker_id in zip(boxes, tracker_ids):
        x1, y1, x2, y2 = map(int, box)
        if tracker_id in tracker_to_reid:
            final_id = tracker_to_reid[tracker_id]

            # --- Detect appearance/reappearance ---
            if final_id not in last_seen_time:
                appearance_log[final_id].append(("appearance"))
                print(f"[{time.strftime('%H:%M:%S', time.localtime(current_time))}] Person {final_id} appeared.")
                csv_writer.writerow([final_id, "appearance",
                                     time.strftime("%H:%M:%S", time.localtime(current_time))
                                     ])
                csv_file.flush()

            else:
                gap = current_time - last_seen_time[final_id]
                if gap > REAPPEAR_THRESHOLD:
                    appearance_log[final_id].append(("reappearance"))
                    print(f"[{time.strftime('%H:%M:%S', time.localtime(current_time))}] Person {final_id} reappeared after {gap:.1f} sec.")
                    csv_writer.writerow([final_id, "reappearance",
                                         time.strftime("%H:%M:%S", time.localtime(current_time))
                                         ])
                    csv_file.flush()

            last_seen_time[final_id] = current_time
        else:
            final_id = -1  # Pending

        label = f"ID: {final_id}" if final_id != -1 else "Pending"
        cv.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv.putText(frame, label, (x1, y1 - 10),
                   cv.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    # Step 4: FPS counter
    ctime = time.time()
    if ptime != 0:
        fps = 1 / (ctime - ptime)
        cv.putText(frame, f"FPS: {int(fps)}", (10, 30),
                   cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    ptime = ctime

    # Step 5: Show
    cv.imshow("Person Re-ID Tracking", frame)
    if cv.waitKey(1) == ord('q'):
        break

# --- Cleanup ---
csv_file.close()
cv.destroyAllWindows()



video 1/1 (frame 1/329) c:\CCTV\palace.mp4: 384x640 10 persons, 107.1ms
video 1/1 (frame 2/329) c:\CCTV\palace.mp4: 384x640 10 persons, 20.6ms
video 1/1 (frame 3/329) c:\CCTV\palace.mp4: 384x640 10 persons, 9.3ms
video 1/1 (frame 4/329) c:\CCTV\palace.mp4: 384x640 10 persons, 8.8ms
video 1/1 (frame 5/329) c:\CCTV\palace.mp4: 384x640 10 persons, 13.1ms
video 1/1 (frame 6/329) c:\CCTV\palace.mp4: 384x640 10 persons, 7.2ms
video 1/1 (frame 7/329) c:\CCTV\palace.mp4: 384x640 10 persons, 8.4ms
video 1/1 (frame 8/329) c:\CCTV\palace.mp4: 384x640 10 persons, 6.2ms
[02:16:07] Person 0 appeared.
[02:16:07] Person 1 appeared.
[02:16:07] Person 2 appeared.
[02:16:07] Person 3 appeared.
[02:16:07] Person 4 appeared.
[02:16:07] Person 5 appeared.
[02:16:07] Person 6 appeared.
[02:16:07] Person 7 appeared.
[02:16:07] Person 8 appeared.
[02:16:07] Person 9 appeared.
video 1/1 (frame 9/329) c:\CCTV\palace.mp4: 384x640 10 persons, 6.8ms
video 1/1 (frame 10/329) c:\CCTV\palace.mp4: 384x640 12 persons, 